<a href="https://colab.research.google.com/github/megha-eng-web/ibm-workin-AI-agent/blob/main/IBM_SkillsBuild_Applied_Artificial_Intelligence_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import pipeline
import datetime

# Load LLM
llm = pipeline(
    "text2text-generation",
    model="google/flan-t5-large",
    max_length=256
)

Device set to use cpu


In [ ]:
print("Atomic Habits AI Agent (type 'exit' to quit)")

# ---------------- MEMORY ---------------- #
agent_memory = {
    "goal": None,
    "habit": None,
    "stage": "discovery",     # discovery → planning → action → review
    "last_action": None,
    "feedback": [],
    "books": []
}

# ---------------- TOOLS ---------------- #

def atomic_habits_strategy(habit, stage):
    prompt = f"""
You are an Atomic Habits expert.
Habit: {habit}
Stage: {stage}

Convert principles into ONE concrete, practical action.
Avoid vague advice.
"""
    return llm(prompt)[0]["generated_text"].strip()


def question_tool():
    prompt = """
Ask ONE specific question to understand the user's habit problem better.
"""
    return llm(prompt)[0]["generated_text"].strip()


def detect_user_intent(user_input):
    prompt = f"""
User says: {user_input}

Classify intent:
CLARIFICATION
FAILED_ATTEMPT
PROGRESS_UPDATE
NEW_INFO

Return ONE label only.
"""
    return llm(prompt)[0]["generated_text"].strip()


def situation_analyzer(user_input):
    prompt = f"""
Analyze emotional situation:
User says: {user_input}

Classify as:
STRESS
RELAPSE
LOW_MOTIVATION
NEUTRAL

Return ONE label only.
"""
    return llm(prompt)[0]["generated_text"].strip()


def coping_strategy_tool(situation):
    prompt = f"""
User situation: {situation}

Give:
1 calming reassurance
1 grounding technique
1 very small next step
"""
    return llm(prompt)[0]["generated_text"].strip()


def book_recommender(habit, situation):
    prompt = f"""
Recommend ONE specific book.
Habit: {habit}
Situation: {situation}

Explain in 2 lines why it helps now.
"""
    return llm(prompt)[0]["generated_text"].strip()


def explain_last_action():
    prompt = f"""
Explain this action clearly with examples:
{agent_memory['last_action']}
"""
    return llm(prompt)[0]["generated_text"].strip()


def adapt_strategy():
    prompt = f"""
User feedback:
{agent_memory['feedback']}

Habit: {agent_memory['habit']}

Generate a better next action avoiding past failure.
"""
    return llm(prompt)[0]["generated_text"].strip()

# ---------------- AGENT CORE ---------------- #

def agent_loop(user_input):
    global agent_memory

    intent = detect_user_intent(user_input)
    situation = situation_analyzer(user_input)

    # ---- Handle clarification ----
    if intent == "CLARIFICATION" and agent_memory["last_action"]:
        return explain_last_action()

    # ---- Handle relapse / stress ----
    if situation in ["STRESS", "RELAPSE"]:
        agent_memory["feedback"].append(user_input)
        return coping_strategy_tool(situation)

    # ---- Discovery stage ----
    if agent_memory["stage"] == "discovery":
        agent_memory["habit"] = user_input
        agent_memory["stage"] = "planning"
        return question_tool()

    # ---- Planning stage ----
    if agent_memory["stage"] == "planning":
        action = atomic_habits_strategy(agent_memory["habit"], "planning")
        agent_memory["last_action"] = action
        agent_memory["stage"] = "action"
        return action

    # ---- Action stage ----
    if agent_memory["stage"] == "action":
        agent_memory["stage"] = "review"
        return "Did you try the action? What happened?"

    # ---- Review stage ----
    if agent_memory["stage"] == "review":
        if intent == "FAILED_ATTEMPT":
            agent_memory["feedback"].append(user_input)
            improved_action = adapt_strategy()
            agent_memory["last_action"] = improved_action
            agent_memory["stage"] = "action"
            return improved_action

        # Suggest book if motivation is low
        if situation == "LOW_MOTIVATION":
            book = book_recommender(agent_memory["habit"], situation)
            agent_memory["books"].append(book)
            return book

        agent_memory["stage"] = "action"
        return atomic_habits_strategy(agent_memory["habit"], "review")


# ---------------- RUN LOOP ---------------- #

while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        print("Agent: Keep building small habits. Goodbye!")
        break

    response = agent_loop(user_input)
    print("Agent:", response)



Atomic Habits AI Agent (type 'exit' to quit)
Agent: What is the user's habit problem?
Agent: If you want to build a good habit, you need to know how to do it.
Agent: Did you try the action? What happened?
Agent: If you want to build a good habit, you need to know how to do it.
Agent: Make a list of all the things you want to do in the future. Write down all the things you want to do in the future. Write down all the things you want to do in the future. Write down all the things you want to do in the future. Write down all the things you want to do in the future. Write down all the things you want to do in the future. Write down all the things you want to do in the future. Write down all the things you want to do in the future. Write down all the things you want to do in the future. Write down all the things you want to do in the future. Write down all the things you want to do in the future. Write down all the things you want to do in the future. Write down all the things you want to d